In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import  tqdm
import string
import urllib

In [2]:
def drop_duplicates_keep_na(df, col='doi'):
    """ Drops duplicates considering NA != NA (NAs are not duplicates)

    Args:
        df: DataFrame to perform the deduplication on
        col: The column to perform the deduplication on

    Returns:
        DataFrame deduplicated on col
    """

    return df[(~df.duplicated(subset=[col])) | (df[col].isnull())]

def drop_duplicates_merge_disorders(df, dedup_col='doi', relevance_suffix='_included_corrected', sort_by='abstract_len'):
    """ Drop dubplicating rows by aggregating indicators of their relevance.

        NOTE: Currently the code implementing aggregation for relevance is buggy, so it is commented out.

    Args:
        df: DataFrame to perform the deduplication on
        dedup_col: Name of a column to perform the deduplication on
        relevance_suffix: Suffix of the columns indicating the relevance of the paper
        sort_by: On which column to sort the rows (firt duplicate row is left)
    
    Returns:
        DataFrame with rows deduplicated on dedup_col, the one remaining row for each set of duplicates contains the relevance
            indicators based on all other duplicates in the set.
        DataFrame with identified potential duplicates (for checking the performance).
    """

    
    dedupDF = df.copy()

    ## Here starts the code to implement the aggregation of relevance columns for duplicates. It is buggy, so is not
    ## turned on yet
    
    # disorders = disorders = ['depression', 'substance', 'anxiety']
    # disorder_cols = [col + relevance_suffix for col in disorders]
    # reviewed_cols = ['reviewed_for_' + d for d in disorders]

    # for disorder, col in zip(disorders, disorder_cols):
    #     dedupDF['reviewed_for_' + disorder] = ~doiFixedDF[col].isna()

    # # Creating the indicator dataframe of whether the paper (groupby row) was reviewed for a disorder (col)
    # revIndicatorDF = dedupDF.groupby(dedup_col).any()[reviewed_cols]
    # # print(revIndicatorDF)

    # # Substituting NA for those papers for which none of duplicates was reviewed for a disorder
    # revIndicatorDF[revIndicatorDF == False] = np.nan
    # # Merging relevance and review indicators for papers
    # indicatorDF = dedupDF.groupby(dedup_col).any()[disorder_cols] * revIndicatorDF.values

    # # print(indicatorDF.shape, dedupDF.shape)
    
    # # Merging the aggregate relevance indicators into each row of the duplicate set
    # dedupDF = dedupDF.merge(indicatorDF[disorder_cols], on=dedup_col, how='left', suffixes=['_old', ''])

    # Sorting before droping the duplicates
    dedupDF.sort_values(sort_by, ascending=False)

    dedupGrouped = dedupDF.groupby(dedup_col)
    dedupDF['group_id'] = dedupGrouped.ngroup()
    dedupDF['group_size'] = dedupGrouped[dedup_col].transform('count')

    dupDF = dedupDF[(dedupDF['group_size'] > 1)].copy().sort_values(['group_size', 'group_id'], ascending=False)
    
    return drop_duplicates_keep_na(dedupDF[df.columns], dedup_col), dupDF

In [3]:
# Reading the merged results file
results_file_path = "megemeta_merged_after_screening_asreview_preliminary.xlsx"
resultsDF = pd.read_excel(results_file_path)

In [4]:
# Getting the data frame of relevant papers
relevantDF = resultsDF[resultsDF.composite_label_corrected > 0].copy()

# Trying to find DOIs through Crossref

In [5]:
# Define Crossref DOI finding function
def crossref_doi_find(row):
    """ Requests Crossref with title-year combination and returns DOI if good
        enough match is found.

    Args:
        row: DataFrame row containing year and title.

    Returns:
        string: Probable DOI of a paper
        None: If no good DOI could be found
    """

    headers = {"Accept": "application/json"}
    year = str(int(row.year))
    url = 'https://api.crossref.org/works/?query.title=' + urllib.parse.quote(row.title) + \
        '&filter=from-pub-date:' + year + ',until-pub-date:' + year
    r = requests.get(url, headers=headers)
    first_entry = r.json()['message']['items'][0]
    title, found_title = [s.translate(string.punctuation).lower() for s in [row.title, first_entry['title'][0]]]
    perfect_match = (title in found_title) or (found_title in title)
    if perfect_match:
        return first_entry["DOI"].lower()

    return None

# Set here the dataframe, for which you want to find missing DOIs
doiFixedDF = relevantDF.copy()
missing_doi_count = doiFixedDF.doi.isna().sum()
print("Requesting Crossref to infer %d missing DOIs" % missing_doi_count)
for i, row in tqdm(doiFixedDF[doiFixedDF.doi.isna()].iterrows()):
    doiFixedDF.loc[i, 'doi'] = crossref_doi_find(row)

fixed_doi_count = missing_doi_count - doiFixedDF.doi.isna().sum()
print("Of %d initially missing DOIs %d (%.2f%%) are found" % (missing_doi_count, fixed_doi_count, 100 * fixed_doi_count/missing_doi_count))

Requesting Crossref to infer 478 missing DOIs


478it [07:34,  1.05it/s]

Of 478 initially missing DOIs 309 (64.64%) are found


In [6]:
print("Of %d initially missing DOIs %d (%.2f%%) are found" % (missing_doi_count, fixed_doi_count, 100 * fixed_doi_count/missing_doi_count))

Of 478 initially missing DOIs 309 (64.64%) are found


## Fixing Unicode in DOIs and saving the file with fixed DOIs

In [7]:
# Fixing the Unicode encoding in some of DOIs
doiFixedDF.loc[~doiFixedDF.doi.isna(), 'doi'] = doiFixedDF[~doiFixedDF.doi.isna()].doi.apply(urllib.parse.unquote)

# Saving the fixed DOI file
doiFixedDF.to_excel('doi_fixed.xlsx', index=False)

# Running the deduplication steps

Reading the file with fixed DOIs

In [8]:
doiFixedDF = pd.read_excel('doi_fixed.xlsx')

## Deduplicating based on DOI

In [9]:
doiFixedDF['abstract_len'] = doiFixedDF.abstract.str.len()
dedupDF, dupDF = drop_duplicates_merge_disorders(doiFixedDF)
print('Number of papers after deduplicating on DOI: %d -> %d' % (len(doiFixedDF), len(dedupDF)))

print('Found duplicates:')
dupDF[['doi', 'title', 'group_id', 'group_size', 'depression_included_corrected', 'anxiety_included_corrected', 'substance_included_corrected', 'composite_label_corrected']]

Number of papers after deduplicating on DOI: 6531 -> 6368
Found duplicates:


,doi,title,group_id,group_size,depression_included_corrected,anxiety_included_corrected,substance_included_corrected,composite_label_corrected
3431,10.1177/070674371105601107,A longitudinal study of risk factors for incid...,5153,3.0,NaN,NaN,1.0,1
3994,10.1177/070674371105601107,A Longitudinal study of risk factors for incid...,5153,3.0,NaN,NaN,1.0,1
3995,10.1177/070674371105601107,A longitudinal study of risk factors for incid...,5153,3.0,NaN,NaN,1.0,1
3277,10.1111/j.1360-0443.1996.tb02336.x,What predicts relapse? Prospective testing of ...,4481,3.0,NaN,NaN,1.0,1
4475,10.1111/j.1360-0443.1996.tb02336.x,What predicts relapse? Prospective testing of ...,4481,3.0,NaN,NaN,1.0,1
...,...,...,...,...,...,...,...,...
1005,10.1001/archpsyc.1994.03950050065007,The time course of nonchronic major depressive...,40,2.0,1.0,NaN,NaN,1
538,10.1001/archgenpsychiatry.2012.43,Twenty-year depressive trajectories among olde...,11,2.0,1.0,NaN,NaN,1
543,10.1001/archgenpsychiatry.2012.43,Twenty-year depressive trajectories among olde...,11,2.0,1.0,NaN,NaN,1
4076,10.1001/archgenpsychiatry.2011.2107,Familial confounding of the association betwee...,9,2.0,NaN,NaN,1.0,1


## Deduplicating based on Title-Year combination

In [10]:
dedupDF['title_year'] = dedupDF.title.str.translate(string.punctuation).str.lower() + ' ' + dedupDF.year.fillna(0).astype('int').astype(str)
dedupDF, dupDF = drop_duplicates_merge_disorders(dedupDF, 'title_year', sort_by=['doi', 'abstract_len'])
print('Number of papers after deduplicating on title-year: %d' % len(dedupDF))

print('Found duplicates')
dupDF[['doi','title', 'group_id', 'group_size', 'depression_included_corrected', 'anxiety_included_corrected', 'substance_included_corrected', 'composite_label_corrected']]

Number of papers after deduplicating on title-year: 6312
Found duplicates


,doi,title,group_id,group_size,depression_included_corrected,anxiety_included_corrected,substance_included_corrected,composite_label_corrected
4617,NaN,Treatment participation and outcome among prob...,6101,3,NaN,NaN,1.0,1
4618,NaN,Treatment participation and outcome among prob...,6101,3,NaN,NaN,1.0,1
4624,NaN,Treatment participation and outcome among prob...,6101,3,NaN,NaN,1.0,1
1191,10.1111/j.0065-1591.2000.acp29[dash]03.x,Why do people with anxiety disorders become de...,6270,2,1.0,NaN,NaN,1
1193,10.1111/j.0065-1591.2000.acp29-03.x,Why do people with anxiety disorders become de...,6270,2,1.0,1.0,NaN,1
...,...,...,...,...,...,...,...,...
6249,10.1097/00019442-200501000-00005,Age at onset of generalized anxiety disorder i...,371,2,NaN,1.0,NaN,1
4874,10.1046/j.1360-0443.1996.91811275.x,A controlled field trial of group versus indiv...,90,2,NaN,NaN,1.0,1
4875,10.1080/09652149639691,A controlled field trial of group versus indiv...,90,2,NaN,NaN,1.0,1
1104,10.1097/00004583-199903000-00012,A clinical trial for adolescent depression: Pr...,71,2,1.0,NaN,NaN,1


In [11]:
dedupDF.to_excel('dedup.xlsx', index=False, columns=relevantDF.columns)